# Core Base Classes

> Base classes and types for all daisyUI components

In [ ]:
#| default_exp core.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union, Literal
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from cjm_tailwind_utils.all import TailwindBuilder
from cjm_fasthtml_daisyui.core.colors import (
    SemanticColor, ColorUtility, ColorBuilder, ColorMixin,
    apply_semantic_colors, get_color_classes
)

## Type Definitions

Common type aliases and enums for daisyUI components.

In [ ]:
#| export
class DaisyPosition(str, Enum):
    """Common position values."""
    TOP = "top"
    BOTTOM = "bottom"
    LEFT = "left"
    RIGHT = "right"
    START = "start"
    CENTER = "center"
    END = "end"
    MIDDLE = "middle"

In [ ]:
#| export
class DaisyBreakpoint(str, Enum):
    """Responsive breakpoints."""
    SM = "sm"
    MD = "md"
    LG = "lg"
    XL = "xl"
    XXL = "2xl"

In [ ]:
#| export
class DaisySize(str, Enum):
    """Common size variants across components."""
    XS = "xs"
    SM = "sm"
    MD = "md"
    LG = "lg"
    XL = "xl"

## Base Component Class

The fundamental class that all daisyUI components inherit from.

In [ ]:
#| export
@dataclass
class DaisyComponent(ColorMixin):
    """Base class for all daisyUI components.
    
    This class provides the foundation for building daisyUI components with:
    - Type-safe semantic color support with automatic content colors
    - Custom class and attribute support
    - Integration with cjm-tailwind-utils for additional styling
    - Responsive modifier support
    """
    
    # HTML attributes
    id: Optional[str] = None
    cls: Optional[str] = None  # Additional custom classes
    attrs: Dict[str, Any] = field(default_factory=dict)
    
    # Common modifiers
    color: Optional[Union[SemanticColor, str]] = None
    size: Optional[Union[DaisySize, str]] = None
    glass: bool = False  # Glass effect modifier
    
    # Responsive modifiers (e.g., {"md": "lg", "lg": "xl"} for responsive sizes)
    responsive_size: Optional[Dict[str, str]] = None
    responsive_hide: Optional[List[str]] = None  # Breakpoints to hide at
    responsive_show: Optional[List[str]] = None  # Breakpoints to show at
    
    # Additional Tailwind customization
    tw_padding: Optional[Union[int, str]] = None
    tw_margin: Optional[Union[int, str]] = None
    tw_utilities: Optional[List[str]] = None  # Raw Tailwind utilities
    
    def component_class(
        self
    ) -> str:  # TODO: Add return description
        """Return the base component class name (e.g., 'btn', 'card')."""
        raise NotImplementedError("Subclasses must implement component_class()")
    
    def modifier_classes(
        self
    ) -> List[str]:  # TODO: Add return description
        """Return all modifier classes for this component."""
        classes = []
        
        # Add color modifier if applicable
        if self.color and self.supports_color():
            color_val = self.color.value if isinstance(self.color, SemanticColor) else self.color
            classes.append(f"{self.component_class()}-{color_val}")
            
        # Add size modifier if applicable
        if self.size and self.supports_size():
            size_val = self.size.value if isinstance(self.size, DaisySize) else self.size
            classes.append(f"{self.component_class()}-{size_val}")
            
        # Add glass modifier
        if self.glass and self.supports_glass():
            classes.append("glass")
            
        # Add responsive size modifiers
        if self.responsive_size and self.supports_size():
            for breakpoint, size in self.responsive_size.items():
                classes.append(f"{breakpoint}:{self.component_class()}-{size}")
                
        return classes
    
    def supports_color(
        self
    ) -> bool:  # TODO: Add return description
        """Whether this component supports color modifiers."""
        return False
    
    def supports_size(
        self
    ) -> bool:  # TODO: Add return description
        """Whether this component supports size modifiers."""
        return False
        
    def supports_glass(
        self
    ) -> bool:  # TODO: Add return description
        """Whether this component supports glass effect."""
        return False
    
    def build_classes(
        self
    ) -> str:  # TODO: Add return description
        """Build complete class string with deduplication."""
        # Use a set to collect all unique classes
        all_classes = set()
        
        # Add component classes
        all_classes.add(self.component_class())
        all_classes.update(self.modifier_classes())
        
        # Add padding/margin
        tb = TailwindBuilder()
        if self.tw_padding is not None:
            tb.p(self.tw_padding)
        if self.tw_margin is not None:
            tb.m(self.tw_margin)
        
        # Get padding/margin classes from TailwindBuilder
        if tb.build():
            all_classes.update(tb.build().split())
            
        # Add responsive visibility
        if self.responsive_hide:
            for bp in self.responsive_hide:
                all_classes.add(f"{bp}:hidden")
        if self.responsive_show:
            for bp in self.responsive_show:
                all_classes.add(f"{bp}:block")
                
        # Add raw utilities
        if self.tw_utilities:
            for utility in self.tw_utilities:
                # Split in case utility contains multiple classes
                all_classes.update(utility.split())
            
        # Collect CSS classes from all mixins that implement get_css_classes
        # Track which actual methods we've called to avoid duplicates
        called_methods = set()
        
        # Check each class in the MRO for get_css_classes method
        for cls in type(self).__mro__:
            if hasattr(cls, 'get_css_classes'):
                method = getattr(cls, 'get_css_classes')
                # Check if this is a unique method (not inherited)
                method_id = id(method.__func__ if hasattr(method, '__func__') else method)
                if callable(method) and method_id not in called_methods:
                    try:
                        result = method(self)
                        if result:
                            all_classes.update(result)
                        called_methods.add(method_id)
                    except:
                        pass
        
        # Add custom classes last
        if self.cls:
            all_classes.update(self.cls.split())
            
        # Convert set back to list and join
        # Sort to ensure consistent order
        return " ".join(sorted(all_classes))
    
    def render_attrs(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Build all HTML attributes for rendering."""
        attrs = {**self.attrs}
        attrs["class"] = self.build_classes()
        
        if self.id:
            attrs["id"] = self.id
            
        return attrs
        
    def with_utilities(
        self,
        *utilities: str
    ) -> 'DaisyComponent':  # TODO: Add return description
        """Add Tailwind utilities and return self for chaining."""
        if self.tw_utilities is None:
            self.tw_utilities = []
        self.tw_utilities.extend(utilities)
        return self
    
    def with_semantic_colors(
        self,
        bg: Optional[Union[SemanticColor, str]] = None,
        text: Optional[Union[SemanticColor, str]] = None,
        border: Optional[Union[SemanticColor, str]] = None,
        auto_content: bool = True  # TODO: Add description
    ) -> 'DaisyComponent':  # TODO: Add return description
        """Apply semantic colors with automatic content color selection.
        
        Args:
            bg: Background color
            text: Text color (auto-selected if None and auto_content=True)
            border: Border color  
            auto_content: Automatically select appropriate text color for background
            
        Returns:
            Self for method chaining
        """
        classes = apply_semantic_colors(bg, text, border, auto_content).split()
        return self.with_utilities(*classes)

## Semantic Color Integration

DaisyComponent now inherits from ColorMixin, providing powerful semantic color support:

- Use `SemanticColor` enum for type-safe color selection
- Automatic content color selection for accessibility
- Methods like `with_brand_colors()` and `with_state_colors()` for quick styling
- Full integration with daisyUI's theme system

In [ ]:
# Example: Creating a simple component with semantic colors
@dataclass
class ExampleComponent(DaisyComponent):
    def component_class(self) -> str:
        return "example"
    
    def supports_color(self) -> bool:
        return True

# Debug the MRO for ExampleComponent
print("MRO for ExampleComponent:")
for i, cls in enumerate(ExampleComponent.__mro__):
    print(f"  {i}: {cls.__name__}")
    if hasattr(cls, 'get_css_classes'):
        print(f"     - has get_css_classes method")

print("\n" + "="*50 + "\n")

# Method 1: Using the color property (for component-specific color classes like btn-primary)
example1 = ExampleComponent(color=SemanticColor.PRIMARY)
print("Method 1 - Color property:")
print(f"  Classes: {example1.build_classes()}")
print(f"  Color: {example1.color}")

# Method 2: Using color methods from ColorMixin
example2 = ExampleComponent()
example2.with_brand_colors("primary")  # Sets bg-primary and text-primary-content
print("\nMethod 2 - Brand colors:")
print(f"  Classes: {example2.build_classes()}")
print(f"  Color classes: {example2.get_css_classes()}")

# Method 3: Using semantic color helper
example3 = ExampleComponent()
example3.with_semantic_colors(
    bg=SemanticColor.SUCCESS,
    border=SemanticColor.SUCCESS,
    auto_content=True  # Automatically adds text-success-content
)
print("\nMethod 3 - Semantic colors:")
print(f"  Classes: {example3.build_classes()}")
print(f"  Utilities: {example3.tw_utilities}")

MRO for ExampleComponent:
  0: ExampleComponent
     - has get_css_classes method
  1: DaisyComponent
     - has get_css_classes method
  2: ColorMixin
     - has get_css_classes method
  3: CSSContributor
     - has get_css_classes method
  4: Protocol
  5: Generic
  6: object


Method 1 - Color property:
  Classes: example example-primary
  Color: SemanticColor.PRIMARY

Method 2 - Brand colors:
  Classes: bg-primary example text-primary-content
  Color classes: ['bg-primary', 'text-primary-content']

Method 3 - Semantic colors:
  Classes: bg-success border-success example text-success-content
  Utilities: ['bg-success', 'text-success-content', 'border-success']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()